<a href="https://colab.research.google.com/github/lumasg/Chatbot/blob/main/modelo_clasificador__de_textos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Configurar ambiente

In [ ]:
#Instalando bibliotecas
import pandas as pd
import re, os, random, pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
!python -m spacy download es_core_news_md
!pip install jellyfish
import jellyfish
!pip install transformers
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
import torch
!pip install python-docx
from docx import Document
import csv
import nltk
nltk.download('punkt')

#Definiendo variables del proyecto:
nlp = spacy.load('es_core_news_md')

#Conectando al Google Drive
from google.colab import drive
drive.mount('drive')
folder = 'drive/MyDrive/Data_Sciencie/Chatbot'

2023-10-03 19:45:00.742758: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 MB 14.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_md')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 41.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184487 sha25

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Mounted at drive


# 2. Importar verbos

In [ ]:
lista_verbos = "drive/MyDrive/Data_Sciencie/Chatbot/verbos/lista_verbos.pickle"
lista_verbos_irregulares = "drive/MyDrive/Data_Sciencie/Chatbot/verbos/verbos_irregulares.pickle"

In [ ]:
# Importar la lista_verbos:
with open(lista_verbos, 'rb') as verbos:
  lista_verbos = pickle.load(verbos)

# Importar el diccionario:
with open(lista_verbos_irregulares, 'rb') as verbosi:
  lista_verbos_irregulares = pickle.load(verbosi)

In [ ]:
print(lista_verbos)
print("\n")
print(lista_verbos_irregulares)

['parar', 'recomendar', 'cancelar', 'fanatizar', 'amaran o amasen', 'exponer', 'obedecer', 'quejar', 'echar', 'legitimar', 'perjudicar', 'organizar', 'molar', 'objetar', 'considerar', 'golear', 'mover', 'acertar', 'reunir', 'regir', 'ilusionar', 'simpatizar', 'conjeturar', 'helar', 'quitar', 'amariamos', 'destacar', 'llegar', 'sincronizar', 'lesionar', 'seducir', 'asistir', 'conservar', 'acordar', 'salvar', 'relucir', 'graduar', 'forzar', 'dar', 'deplorar', 'batear', 'mofar', 'estropear', 'aplastar', 'wasapeo', 'gestionar', 'suprimir', 'gruñir', 'progresar', 'suscribir', 'noticiar', 'cavar', 'alejar', 'galopar', 'virar', 'medir', 'actualizar', 'humanizar', 'convivir', 'gratificar', 'digerir', 'tocar', 'zonificar', 'amariais', 'aterrizar', 'hojear', 'cometer', 'sufrir', 'reciclar', 'obturar', 'divertir', 'ondear', 'listar', 'determinar', 'alentar', 'sumar', 'reflexionar', 'ames', 'anotar', 'mitificar', 'escribir', 'ilustrar', 'obtener', 'subir', 'socorrer', 'desprender', 'agregar', 'gan

#3. Tratamiento de datos

In [ ]:
#Función para encontrar la raiz de las palabras
def raiz(palabra):
    max_similitud = 0.0
    for verbo in lista_verbos:
        similitud = jellyfish.jaro_similarity(palabra, verbo)
        if similitud > max_similitud:
            max_similitud = similitud
            palabra_encontrada = verbo

    if max_similitud >= 0.93:
        return palabra_encontrada
    else:
        return palabra

def tratamiento_texto(texto):
  trans = str.maketrans('áéíóúâà','aeiouaa')
  texto = texto.lower()
  texto = texto.translate(trans)
  texto = re.sub(r"[^\w\s+\-*/¿?¡!]", '', texto)
  texto = " ".join(texto.split())
  # print('texto de tratamiento_texto: ',texto)
  return texto

#Función para reemplazar el final de una palabra por 'r'
def reemplazar_terminacion(frase):
    terminaciones = ["es", "me", "as", "ste", "te"]
    palabras = frase.split()

    for i, palabra in enumerate(palabras):
        for terminacion in terminaciones:
            if palabra.endswith(terminacion) and len(palabra) > len(terminacion):
                palabras[i] = palabra[:-len(terminacion)] + "r"
                break

    nueva_frase = " ".join(palabras)
    return nueva_frase

#Función para adicionar o eliminar tokens
def revisar_tokens(texto, tokens):
  texto=tratamiento_texto(texto)
  if len(tokens)==0:
    if any(name in texto for name in ['cientifico de datos', 'data scientist']):
        tokens.append('datascientist')
    if any(name in texto for name in ['elprofealejo', 'el profe alejo', 'profe alejo', 'profealejo']):
        tokens.append('elprofealejo')
    if any(name in texto for name in ['ciencia de datos', 'data science']):
        tokens.append('datascience')
  else:
    elementos_a_eliminar = ["cual", "que", "quien", "cuanto", "cuando", "como"]
    if 'hablame' in texto and 'hablar' in tokens: elementos_a_eliminar.append('hablar')
    elif 'cuentame' in texto and 'contar' in tokens: elementos_a_eliminar.append('contar')
    elif 'hago' in texto and 'hacer' in tokens: elementos_a_eliminar.append('hacer')
    elif 'entiendes' in texto and 'entender' in tokens: elementos_a_eliminar.append('entender')
    elif 'sabes' in texto and 'saber' in tokens: elementos_a_eliminar.append('saber')
    tokens = [token for token in tokens if token not in elementos_a_eliminar]
  return tokens

#Función para devolver los tokens normalizados del texto
def normalizar(texto):
  tokens=[]
  doc = nlp(texto)
  for t in doc:
    lemma=lista_verbos_irregulares.get(t.text, t.lemma_.split()[0])
    lemma=re.sub(r'[^\w\s+\-*/]', '', lemma)
    if t.pos_ in ('VERB','PROPN','PRON','NOUN','AUX','SCONJ','ADJ','ADV','NUM') or lemma in lista_verbos:
      if t.pos_=='VERB':
        lemma = reemplazar_terminacion(lemma)
        tokens.append(raiz(tratamiento_texto(lemma)))
      else:
        tokens.append(tratamiento_texto(lemma))

  tokens = list(dict.fromkeys(tokens))
  tokens = list(filter(None, tokens))
  cadena_texto = ' '.join(str(x) for x in tokens)
  return cadena_texto
  # return tokens

In [ ]:
#Función para devolver los tokens normalizados del texto
def normalizar2(texto):
    tokens = []
    tokens = revisar_tokens(texto, tokens)
    doc = nlp(texto)
    for t in doc:
        # Obtener el lemma
        lemma = lista_verbos_irregulares.get(t.text, t.lemma_)
        #lemma = lista_verbos_irregulares.get(t.text, t.lemma_.split()[0])

        # Verificar si lemma es una cadena no vacía
        if lemma and isinstance(lemma, str):
            lemma = re.sub(r'[^\w\s+\-*/]', '', lemma)

            if t.pos_ in ('VERB','PROPN','PRON','NOUN','AUX','SCONJ','ADJ','ADV','NUM') or lemma in lista_verbos:
                if t.pos_ == 'VERB':
                    lemma = reemplazar_terminacion(lemma)
                    tokens.append(raiz(tratamiento_texto(lemma)))
                else:
                    tokens.append(tratamiento_texto(lemma))

    tokens = list(dict.fromkeys(tokens))
    tokens = list(filter(None, tokens))
    tokens = revisar_tokens(texto, tokens)
    tokens_str = str(tokens)
    # print('tokens: ',tokens_str)
    return tokens_str

In [ ]:
frase='Como ESTas    AMIgo Mio?'
texto_tratado=tratamiento_texto(frase)
print(texto_tratado)

como estas amigo mio?


In [ ]:
normalizar('Como ESTAS AMIgo Mio?')

'como amigo mio'

# 4. Cargar bases de documentos

In [ ]:
#Importando bases de dialogo

txt_folder_path = folder + '/dialogos'
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".txt")]
lista_dialogos, lista_dialogos_respuesta, lista_tipo_dialogo = [],[],[]

for idx in range(len(lista_documentos)):
  f=open(txt_folder_path+'/'+lista_documentos[idx], 'r', encoding='utf-8', errors='ignore')
  estado = True
  for line in f.read().split('\n'):
    if estado:
      line_tratado = tratamiento_texto(line)
      lista_dialogos.append(line_tratado)
      lista_tipo_dialogo.append(lista_documentos[idx][:-4])
    else:
      lista_dialogos_respuesta.append(line)
    estado=not estado

#Creando Dataframe de diálogos
datos = {'dialogo':lista_dialogos,'respuesta':lista_dialogos_respuesta,'tipo':lista_tipo_dialogo,'interseccion':0,'jaro_winkler':0,'probabilidad':0}
df_dialogo = pd.DataFrame(datos)
df_dialogo = df_dialogo.drop_duplicates(keep='first')
df_dialogo.reset_index(drop=True, inplace=True)

#Importando bases csv
txt_folder_path = folder+'/documentos'
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".csv")]
documento_csv = ''
for i in range(len(lista_documentos)):
  with open(txt_folder_path+'/'+lista_documentos[i], "r", encoding="utf-8") as csv_txt:
    csv_text = csv.reader(csv_txt)
    for fila in csv_text:
      if fila[-1]!='frase':
        documento_csv += fila[-1]

#Importando bases docx
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".docx")]
documento_docx = ''
for i in range(len(lista_documentos)):
  for t in Document(txt_folder_path+'/'+lista_documentos[i]).paragraphs:
    documento_docx += t.text.replace('*','\n\n*')+ "\n"

#Importando bases txt
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".txt")]
documento_txt = ''

for i in range(len(lista_documentos)):
  with open(txt_folder_path+'/'+lista_documentos[i], "r", encoding="utf-8") as txt:
    txt_new = txt.read()
    for i in txt_new:
      documento_txt += i

In [ ]:
df_dialogo

,dialogo,respuesta,tipo,interseccion,jaro_winkler,probabilidad
0,ahora quien responde,Yo respondo.,Identidad,0,0,0
1,ahora ya no se ni con quien o que estoy hablando,"Lo siento, no entendí tu pregunta. ¿Puedes exp...",Identidad,0,0,0
2,ahorita estoy chateando con una ia,¡Sí soy una Inteligencia Artificial llamada Ma...,Identidad,0,0,0
3,bien quien eres,"¡Hola! Me llamo Madre, ¡el chatbot más guay de...",Identidad,0,0,0
4,el chat bot o la persona,¡Sí soy una Inteligencia Artificial llamada Ma...,Identidad,0,0,0
...,...,...,...,...,...,...
1147,estoy agradecido,Es maravilloso escuchar que te sientes agradec...,Sentimiento,0,0,0
1148,me siento ansioso,Lamento que estés experimentando ansiedad. Si ...,Sentimiento,0,0,0
1149,estoy confundido,Entiendo cómo te sientes. La confusión puede s...,Sentimiento,0,0,0
1150,me siento inspirado,¡Qué genial que te sientas inspirado! La inspi...,Sentimiento,0,0,0


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

#Normalizando las frases
label_encoder = LabelEncoder()
df_dialogo['palabras'] = df_dialogo['dialogo'].apply(lambda x: normalizar(x))
#df_dialogo['palabras'] = df_dialogo['dialogo'].apply(lambda x: ' '.join(normalizar(x)))
df_dialogo['tipo_num'] = label_encoder.fit_transform(df_dialogo['tipo'])
df_dialogo = df_dialogo[df_dialogo.palabras.values!='']
df_dialogo

,dialogo,respuesta,tipo,interseccion,jaro_winkler,probabilidad,palabras,tipo_num
0,ahora quien responde,Yo respondo.,Identidad,0,0,0,ahora quien responder,9
1,ahora ya no se ni con quien o que estoy hablando,"Lo siento, no entendí tu pregunta. ¿Puedes exp...",Identidad,0,0,0,ahora ya no el quien que estar hablar,9
2,ahorita estoy chateando con una ia,¡Sí soy una Inteligencia Artificial llamada Ma...,Identidad,0,0,0,ahorita estar chatear ia,9
3,bien quien eres,"¡Hola! Me llamo Madre, ¡el chatbot más guay de...",Identidad,0,0,0,bien quien ser,9
4,el chat bot o la persona,¡Sí soy una Inteligencia Artificial llamada Ma...,Identidad,0,0,0,chat bot persona,9
...,...,...,...,...,...,...,...,...
1147,estoy agradecido,Es maravilloso escuchar que te sientes agradec...,Sentimiento,0,0,0,estar agradecido,14
1148,me siento ansioso,Lamento que estés experimentando ansiedad. Si ...,Sentimiento,0,0,0,yo sentir ansioso,14
1149,estoy confundido,Entiendo cómo te sientes. La confusión puede s...,Sentimiento,0,0,0,estar confundido,14
1150,me siento inspirado,¡Qué genial que te sientas inspirado! La inspi...,Sentimiento,0,0,0,yo sentir inspirado,14


##Modelo con Transformers

In [ ]:
# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
df_train, df_test = train_test_split(df_dialogo, test_size=0.2, random_state=42)

# Cargar el modelo preentrenado de BERT para clasificación en español
model_name = 'dccuchile/bert-base-spanish-wwm-uncased'
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=df_dialogo['tipo_num'].nunique())
tokenizer = BertTokenizer.from_pretrained(model_name)

# Tokenizar y codificar las frases de entrenamiento
train_inputs = tokenizer.batch_encode_plus(
    df_train['palabras'].tolist(),
    max_length=128,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

# Tokenizar y codificar las frases de prueba
test_inputs = tokenizer.batch_encode_plus(
    df_test['palabras'].tolist(),
    max_length=128,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

# Preparar los datos de entrenamiento y prueba
train_data = torch.utils.data.TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], torch.tensor(df_train['tipo_num'].tolist()))
test_data = torch.utils.data.TensorDataset(test_inputs['input_ids'], test_inputs['attention_mask'], torch.tensor(df_test['tipo_num'].tolist()))

# Definir el optimizador y la función de pérdida
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Entrenamiento del modelo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()

train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)

for epoch in range(5):  # Número de épocas de entrenamiento
    total_loss = 0

    for batch in train_dataloader:
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    print("Epoch:", epoch + 1, "Loss:", total_loss)

# Evaluación del modelo
model.eval()
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)

with torch.no_grad():
    predictions = []
    true_labels = []

    for batch in test_dataloader:
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)

        outputs = model(input_ids, attention_mask=attention_mask)

        _, predicted_labels = torch.max(outputs.logits, dim=1)

        predictions.extend(predicted_labels.tolist())
        true_labels.extend(labels.tolist())

accuracy = accuracy_score(true_labels, predictions)
accuracy=round(accuracy,4)*100
print("Precisión:", accuracy,'%')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['classifier.bias', 'bert.pooler.dense.bias', 'classifier.weight', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 1 Loss: 137.786985039711
Epoch: 2 Loss: 109.66315054893494
Epoch: 3 Loss: 81.69982814788818
Epoch: 4 Loss: 58.10445612668991
Epoch: 5 Loss: 41.0915549993515
Precisión: 75.22 %


In [ ]:
# # Guardar el modelo entrenado
ruta_modelo = 'drive/MyDrive/Data_Sciencie/Chatbot'
model.save_pretrained(ruta_modelo)
tokenizer.save_pretrained(ruta_modelo)

('drive/MyDrive/Data_Sciencie/Chatbot/tokenizer_config.json',
 'drive/MyDrive/Data_Sciencie/Chatbot/special_tokens_map.json',
 'drive/MyDrive/Data_Sciencie/Chatbot/vocab.txt',
 'drive/MyDrive/Data_Sciencie/Chatbot/added_tokens.json')

In [ ]:
#Cargar el modelo entrenado
ruta_modelo = 'drive/MyDrive/Data_Sciencie/Chatbot'
Modelo_TF = BertForSequenceClassification.from_pretrained(ruta_modelo)
tokenizer_TF = BertTokenizer.from_pretrained(ruta_modelo)